In [1]:
import torch

D:\surya\miniconda37\envs\imgseg\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model = torch.hub.load('anibali/segformer', 'segformer_b2', pretrained=True, num_classes=3)

Using cache found in C:\Users\Jayaraman/.cache\torch\hub\anibali_segformer_master
Downloading: "https://github.com/anibali/segformer/releases/download/v0.0.0/segformer_b2_backbone_imagenet-3c162bb8.pth" to C:\Users\Jayaraman/.cache\torch\hub\checkpoints\segformer_b2_backbone_imagenet-3c162bb8.pth
100%|█████████████████████████████████████| 92.4M/92.4M [00:10<00:00, 9.59MB/s]


In [6]:
list(model.state_dict().keys())

['backbone.stages.0.patch_embed.proj.weight',
 'backbone.stages.0.patch_embed.proj.bias',
 'backbone.stages.0.patch_embed.norm.weight',
 'backbone.stages.0.patch_embed.norm.bias',
 'backbone.stages.0.blocks.0.norm1.weight',
 'backbone.stages.0.blocks.0.norm1.bias',
 'backbone.stages.0.blocks.0.attn.q.weight',
 'backbone.stages.0.blocks.0.attn.q.bias',
 'backbone.stages.0.blocks.0.attn.kv.weight',
 'backbone.stages.0.blocks.0.attn.kv.bias',
 'backbone.stages.0.blocks.0.attn.proj.weight',
 'backbone.stages.0.blocks.0.attn.proj.bias',
 'backbone.stages.0.blocks.0.attn.sr.weight',
 'backbone.stages.0.blocks.0.attn.sr.bias',
 'backbone.stages.0.blocks.0.attn.norm.weight',
 'backbone.stages.0.blocks.0.attn.norm.bias',
 'backbone.stages.0.blocks.0.norm2.weight',
 'backbone.stages.0.blocks.0.norm2.bias',
 'backbone.stages.0.blocks.0.ffn.fc1.weight',
 'backbone.stages.0.blocks.0.ffn.fc1.bias',
 'backbone.stages.0.blocks.0.ffn.conv.weight',
 'backbone.stages.0.blocks.0.ffn.conv.bias',
 'backbone

In [11]:
# input = torch.Tensor(1,3, 360, 640)
# output = model(input)
# output.shape

torch.Size([1, 3, 360, 640])